In [1]:
import plotly.graph_objects as go

import json
import networkx as nx

import numpy as np
import base64

In [2]:
G = nx.Graph()

In [3]:
excluded_nodes = [1,2,4,31,32, 34,35,36]

In [4]:
network = json.load(open('../../../../../tsm-weaver-simulator/sim_networks/testbed_disi_network.json'))

for node in network['nodes']:
    if node['id'] not in excluded_nodes:
        G.add_node(node['id'], pos=node['coords'], hoverlabel=node['id'])
    
for node in network['nodes']:
    if node['id'] not in excluded_nodes:
        for next_node in node['adj']:
            if next_node not in excluded_nodes:
                G.add_edge(node['id'], next_node)

In [5]:
node_x = []
node_y = []
node_text = []
for node in G.nodes():
    x, y = G.nodes[node]['pos']
    node_x.append(x)
    node_y.append(y)
    node_text.append(node)

In [6]:
shortest=dict(nx.all_pairs_shortest_path_length(G))

In [7]:
image1_filename = '/home/esoprana/index.jpg'
polar_light = base64.b64encode(open(image1_filename, 'rb').read())

f = go.FigureWidget([go.Scatter(
    x=node_x, 
    y=node_y, 
    text=node_text,
    textposition='top right',
    textfont=dict(color='#E58606'),
    mode='markers+text',
    marker = dict(
        colorscale='turbo',
        colorbar=dict(title='Hops'),
        size=15,
    ),
    hoverinfo='none',
), go.Scatter(
    x=[],
    y=[],
    mode='lines',
    line=dict(
        color='rgba(0, 0, 0, 0.75)',
        width=2,
    )
)],
layout=go.Layout(
    title='Disi network',
    titlefont_size=16,
    showlegend=False,
    hovermode='closest',
    xaxis=dict(showgrid=False, zeroline=False, showticklabels=True),
    yaxis=dict(showgrid=False, zeroline=False, showticklabels=True),

    images= [dict(
        source='data:image/png;base64,{}'.format(polar_light.decode()),
        xref= "x",
        yref= "y",
        x=-20,
        y=-17,
        sizex=114,
        sizey=62,
        xanchor='left',
        yanchor='bottom',
        sizing= "stretch",
        layer='below'
    )],
    width=1100,
    height=600,
))

scatter = f.data[0]
f.layout.hovermode = 'closest'

# create our callback function
def update_point(trace, points, selector):   
    edge_x = []
    edge_y = []
    for edge in G.edges():
        x0, y0 = G.nodes[edge[0]]['pos']
        x1, y1 = G.nodes[edge[1]]['pos']
        
        if len(points.point_inds) < 0:
            pass
        else:
            actual_id = list(set(range(1,37)) - set(excluded_nodes))[points.point_inds[0]]
            
            if actual_id in [edge[0], edge[1]]:
                edge_x.append(x0)
                edge_x.append(x1)
                edge_x.append(None)
                edge_y.append(y0)
                edge_y.append(y1)
                edge_y.append(None)

                f.data[0].marker.color = [v for k, v in sorted(shortest[actual_id].items())] #[i if i== points.point_inds[0] else 'red' for i in range(len(f.data[0].x))]

        
    f.data[1].x = edge_x
    f.data[1].y = edge_y

scatter.on_hover(update_point)

f

FigureWidget({
    'data': [{'hoverinfo': 'none',
              'marker': {'colorbar': {'title': {'text': 'Hop…